In [1]:
import numpy as np
import os
import h5py
import dask.array as da
import dask.bag as db
from collections import Counter
import re
from functools import reduce
import operator as op
import json

C:\pyhton\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Создание dask.array

In [3]:
arr = da.random.normal(0, 1, size = (1000, 10000), chunks = (1000, 1000))

Запись dask.array в файл 

In [8]:
with h5py.File('dask_array.hdf5', 'w') as f:
    f.create_dataset('ds1', data = arr)

Чтение из файла

In [11]:
res  = h5py.File('dask_array.hdf5', 'r')
list(res.keys())

['ds1']

In [12]:
rd = da.from_array(res['ds1'], chunks = (1000, 1000))
rd

dask.array<array, shape=(1000, 10000), dtype=float64, chunksize=(1000, 1000)>

Рассчёт минимального значения массива

In [13]:
da.min(rd).compute()

-5.120207738741429

In [9]:
path = 'D:/all_k/all_k'

In [10]:
b = db.read_text(os.path.join(path , '*.txt'), encoding = 'ansi')

In [11]:
b.npartitions

664

In [12]:
b.take(10)

('         Хулио Кортасар\n',
 '       \n',
 '         62. Модель для сборки\n',
 '       \n',
 '          \n',
 '       \n',
 '         Многие читатели, наверно, заметят, что в этом произведении я  кое-где\n',
 '      преступаю литературные условности. Приведу лишь несколько примеров:  ар-\n',
 '      гентинцы у меня то и дело переходят с "вы" на "ты", когда  это  для  них\n',
 '      естественно в диалоге; житель  Лондона,  только  недавно  бравший  уроки\n')

In [41]:
def count_letters(text):
    text = text.lower()
    txts = re.findall("[а-яА-ЯёЁ]", text)
    return Counter(txts)

In [9]:
b.map(count_letters).fold(op.add).compute()

Counter({'а': 21322436,
         'б': 4860996,
         'в': 11312096,
         'г': 4737949,
         'д': 8105547,
         'е': 21969368,
         'ж': 2845293,
         'з': 4495933,
         'и': 17062720,
         'й': 2911202,
         'к': 8622932,
         'л': 13464478,
         'м': 8455309,
         'н': 17572093,
         'о': 29120773,
         'п': 7335858,
         'р': 11631526,
         'с': 13965456,
         'т': 15853252,
         'у': 7384808,
         'ф': 453087,
         'х': 2189394,
         'ц': 881932,
         'ч': 3853340,
         'ш': 2129603,
         'щ': 898742,
         'ъ': 61751,
         'ы': 4848844,
         'ь': 5177834,
         'э': 1132553,
         'ю': 1510726,
         'я': 5438075,
         'ё': 447183})

In [11]:
rus_symb = set(chr(symb) for symb in range(ord('а'), ord('я')+1))

In [203]:
def count_words(text):
    text = text.split(' ')
    final = [w for w in text if w[0] in rus_symb]
    return Counter(final)

In [16]:
rus_symb = set(chr(symb) for symb in range(ord('а'), ord('я')+1))
def is_cyrillic(word):
    if len(word) > 0:
        return bool((word[0] in rus_symb) * is_cyrillic(word[1:]))
    else: return True

In [17]:
def count_words(text):
    reslist = re.findall('\w+', text.lower())
    return Counter(filter(is_cyrillic, reslist))

In [20]:
b_1 = db.read_text('D:/all_k/all_k/БРАВО.txt', encoding = 'ansi')

In [27]:
b = db.read_text(os.path.join(path , '*.txt'), encoding = 'ansi')

In [21]:
%time
res_1 = b_1.map(count_words).fold(op.add).compute()
print(len(res_1))

Wall time: 0 ns
21335


In [108]:
b_2 = db.read_text('D:/all_k/ПРУДЕНС.txt', encoding = 'iso8859_5')

In [4]:
jbag = db.read_text('datasets/Toys_and_Games_5.json')

In [33]:
def count_json(line):
    if (line['overall'] == 5.0):
        reslist = re.findall('\w+', line['reviewText'].lower())
        return Counter(reslist)
    else: return Counter()

In [31]:
res = jbag.map(json.loads)

In [37]:
type(res)

dask.bag.core.Bag

In [34]:
res1 = res.map(count_json).fold(op.add).compute()

KeyboardInterrupt: 